# Chatting with LangChain by [@ivancampos](https://medium.com/@ivancampos)

![](https://cdn.midjourney.com/758fa4c2-a61b-4693-9c1e-5abcd97b24e5/0_1.png)

In [48]:
!pip install langchain openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
OPENAI_API_KEY = "YOUR OPENAI API KEY"

In [50]:
from langchain.chat_models import ChatOpenAI

ChatGPT = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name="gpt-3.5-turbo"
)

In [51]:
from langchain.schema import HumanMessage

prompt = [
    HumanMessage(content="list three colors"),
]

In [52]:
chat = ChatGPT(prompt)

In [53]:
print(chat)

content='1. Blue\n2. Green\n3. Red' additional_kwargs={}


In [54]:
# Print just the content
print(chat.content)

1. Blue
2. Green
3. Red


# Message Types

The Chat API from OpenAI accepts three Message Types: 
* SystemMessage: Sets the behavior
* AIMessage: Provide examples
* HumanMessage: Set the instructions

In [55]:
from langchain.schema import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="you list items in reverse alphabetical order"),
    HumanMessage(content="list three colors"),
]

# Non-Deterministic
By default, our chat has no memory, so you may be returned three entirely different colors when compared to your previous chat.

In [56]:
chat = ChatGPT(messages)

print(chat.content)

Yellow, white, and violet.


In [57]:
# Imports
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# System Message
system_template="You are a helpful assistant that translates {input_language} to English Boston Accent."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# Human Message
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Prompt
prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# Chat
chat = ChatGPT(prompt.format_prompt(input_language="English", text="Do you want to visit the Harbor or Harvard Yard").to_messages())

# Print
print(chat.content)

Do ya wanna visit the Hahbah or Hahvahd Yahd?


# Searching Text and PDF with Embeddings

In [58]:
!pip install chromadb
from langchain.vectorstores import Chroma

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Chat with a .txt File

In [59]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

!wget -nc https://github.com/IvanCampos/langChain/raw/main/data/us-constitution.txt -O us-constitution.txt
loader = TextLoader("./us-constitution.txt")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
txt_search = Chroma.from_documents(texts, embeddings)


--2023-05-01 02:39:52--  https://github.com/IvanCampos/langChain/raw/main/data/us-constitution.txt
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IvanCampos/langChain/main/data/us-constitution.txt [following]
--2023-05-01 02:39:52--  https://raw.githubusercontent.com/IvanCampos/langChain/main/data/us-constitution.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44840 (44K) [text/plain]
Saving to: ‘us-constitution.txt’

us-constitution.txt 100%[===================>]  43.79K  --.-KB/s    in 0.001s  

2023-05-01 02:39:52 (35.3 MB/s) - ‘us-constitution.txt’ saved [44840/44840]



In [60]:
from langchain.chat_models import ChatOpenAI

ChatGPT = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name="gpt-3.5-turbo"
)

chatTXT = RetrievalQA.from_chain_type(llm=ChatGPT, chain_type="stuff", retriever=txt_search.as_retriever())

prompt = "What is the 2nd ammendment"
chatTXT.run(prompt)

'The 2nd Amendment states that "A well regulated Militia, being necessary to the security of a free State, the right of the people to keep and bear Arms, shall not be infringed."'

# Chat with a PDF

In [61]:
!pip install pypdf tiktoken
from langchain.document_loaders import PyPDFLoader

!wget -nc https://github.com/IvanCampos/langChain/raw/main/data/pkd-metz.pdf -O pkd-metz.pdf
loader = PyPDFLoader("./pkd-metz.pdf")
pages = loader.load_and_split()

embeddings = OpenAIEmbeddings()
pdf_search = Chroma.from_documents(pages, embeddings)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-05-01 02:40:01--  https://github.com/IvanCampos/langChain/raw/main/data/pkd-metz.pdf
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IvanCampos/langChain/main/data/pkd-metz.pdf [following]
--2023-05-01 02:40:01--  https://raw.githubusercontent.com/IvanCampos/langChain/main/data/pkd-metz.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 91382 (89K) [application/octet-stream]
Saving to: ‘pkd-metz.pdf’

pkd-metz.pdf        100%[===================>]  89.24K  --.-KB/s    in 0.002s  

2023-05-0

In [62]:
from langchain.chat_models import ChatOpenAI

ChatGPT = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name="gpt-4"
)

chatPDF = RetrievalQA.from_chain_type(llm=ChatGPT, chain_type="stuff", retriever=pdf_search.as_retriever())

prompt = "are we living in a computer generated simulation"
chatPDF.run(prompt)

'Based on the context provided, the idea of living in a computer-generated simulation is explored through the concept of the "Programmer and Reprogrammer." This concept suggests that there might be an entity responsible for programming and reprogramming our reality, similar to how a computer simulation might be created and altered. However, this idea remains speculative and theoretical, and there is no definitive evidence to prove that we are indeed living in a computer-generated simulation.'